In [2]:
### MOES tests

import numpy as np; from numpy import matlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
path = os.getcwd() # the path to codes_01_energy_demand.py

weather = pd.read_csv(os.path.join(path, "Weather.csv"),header=0,encoding = 'unicode_escape')
weather.columns = ['Temp', 'Irr']

buildings = pd.read_csv(os.path.join(path, "Buildings.csv"),header=0,encoding = 'unicode_escape')
buildings.columns = ['Name', 'Year', 'Ground', 'Heat', 'Elec']
ground = buildings['Ground'].loc[buildings.Name=='BS']
ground

1    10267
Name: Ground, dtype: int64

In [60]:
occ_off=[0,0,0,0,0,0,0,0.2,0.4,0.6,0.8,0.8,0.4,0.6,0.8,0.8,0.4,0.2,0,0,0,0,0,0] # occupancy profile of offices
occ_class=[0,0,0,0,0,0,0,0.4,0.6,1,1,0.8,0.2,0.6,1,0.8,0.8,0.4,0,0,0,0,0,0] #occupancy profile of classrooms
occ_can=[0,0,0,0,0,0,0,0,0.4,0.2,0.4,1,0.4,0.2,0.4,0,0,0,0,0,0,0,0,0] #occupancy profile of canteens
weekday_elec=[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0] # electricity profile of weekdays

occ_all = pd.DataFrame({'Office': occ_off, 'Class': occ_class, 'Restaurant': occ_can, 'Electricity': weekday_elec})
start_date = '2023-01-01'
end_date = '2024-01-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='H', inclusive='right')

occupancy = occ_all # initialize
while len(occupancy) < len(date_range): #extend to a year
    occupancy= pd.concat([occupancy, occ_all], ignore_index=True)

occupancy = occupancy.iloc[0:len(date_range),:]  # drop extra rows 
occupancy['Date'] = pd.DataFrame({'Date': date_range}) 
occupancy['DayType'] = np.where(occupancy['Date'].dt.weekday < 5, 'Weekday', 'Weekend') # add day type
occupancy.loc[occupancy.DayType == 'Weekend', ['Office', 'Class', 'Restaurant', 'Electricity']] = 0 # set weekend occupancy to 0
#occ_year[['Office','Class','Restaurant','Electricity']].where(occ_year['DayType']=='Weekend', 0, inplace=True)
occupancy = occupancy.drop(['Date', 'DayType'], axis=1) # drop date and day type
occupancy.Electricity.sum()

3640

In [66]:
# Define occupancy and electricity profiles
occ_off = [0, 0, 0, 0, 0, 0, 0, 0.2, 0.4, 0.6, 0.8, 0.8, 0.4, 0.6, 0.8, 0.8, 0.4, 0.2, 0, 0, 0, 0, 0, 0]
occ_class = [0, 0, 0, 0, 0, 0, 0, 0.4, 0.6, 1, 1, 0.8, 0.2, 0.6, 1, 0.8, 0.8, 0.4, 0, 0, 0, 0, 0, 0]
occ_can = [0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.2, 0.4, 1, 0.4, 0.2, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
weekday_elec = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

# Create a date range
start_date = '2021-01-01'
end_date = '2022-01-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='H', inclusive='right')

occupancy = occ_all # initialize
while len(occupancy) < len(date_range): #extend to a year
    occupancy= pd.concat([occupancy, occ_all], ignore_index=True)

# Create the occupancy DataFrame
occupancy.index = date_range
occupancy.loc[(date_range.weekday >= 5), ['Office', 'Class', 'Restaurant', 'Electricity']] = 0
#occupancy['DayType'] = np.where(occupancy.index.weekday < 5, 'Weekday', 'Weekend')
#occupancy.loc[occupancy['DayType'] == 'Weekend', ['Office', 'Class', 'Restaurant', 'Electricity']] = 0
#occupancy.drop(['DayType'], axis=1, inplace=True)
#occupancy.reset_index(inplace=True, drop = True)
occupancy.head(50)
occupancy.Electricity.sum()


3654

In [715]:
# Heat gains from people (Office, Restaurant, Classroom)
rooms = pd.DataFrame([{'Office':5, 'Class':23.3, 'Restaurant':35,  'Others':0},{'Office':0.3,'Class':0.35, 'Restaurant':0.05,  'Others':0.3}], index=['Heat gains', 'Area share'])
# Share areas (Office, Restaurant, Classroom)
building_id = 'BS'

q = pd.DataFrame(index=occupancy.index, columns=occupancy.columns) # initialize
for roomType in ['Office', 'Class', 'Restaurant']:
    q[roomType] = occupancy[roomType].values* rooms.loc['Heat gains', roomType] * rooms.loc['Area share', roomType]
# scaling factor per room type
# hourly heat gains per m2 from people according to occupancy for each room type (column name)
#q.drop(['Electricity'], axis=1, inplace=True) # drop electricity
q['All heat'] = q_people.sum(axis=1)
#q_people = occupancy[['Office', 'Class', 'Restaurant']].mul(rooms.loc['Scaling factor',['Office', 'Class', 'Restaurant']]).sum(axis=1) # hourly heat gains per m2 from people according to occupancy    

In [716]:
#Elec gains
cf = 1000/3654 # conversion factor from kWh to W, rescale to operating hours
buildings['Elec/area'] = buildings.apply(lambda x: x.Elec*cf/x.Ground, axis=1) # W/m2 h, hourly elec consumption, by buildings 
q_elec = buildings['Elec/area'].loc[buildings.Name==building_id]# hourly heat gains per m2 from people according to occupancy and room type


q.loc[occupancy['Electricity'] == 1, 'Electricity'] = q_elec.iloc[0]

In [757]:
q=q.fillna(0)
A_th = buildings.Ground.loc[buildings.Name==building_id].values[0]
# m2, floor area
T_int = 21 # degC, internal temperature
T_ext = weather.Temp.iloc[0:len(date_range)] # degC, external temperature
irr = weather.Irr.iloc[0:len(date_range)] # W/m2, solar irradiance
f_el=0.8    # fraction of elec consumption that is converted to heat
T_th = 16

q_elec = q.loc[:,['Electricity']]
q_people = q.loc[:,['All heat']]


cutoff_indicator = ((T_ext >= T_th -1) & (T_ext <= T_th + 1))

#q_elec_mean = q_elec.loc[cutoff_indicator].mean(axis=0)
q_people_mean = q_people[cutoff_indicator].mean()[0]
q_elec_mean = q_elec[cutoff_indicator].mean()[0]
irr_mean = irr[cutoff_indicator].mean()
#Q_th =A_th* (k_th*(T_int-T_ext)- k_sun*irr -q_elec*f_el) # W, hourly heat gains from elec and heat transfer through the envelope

print(q_people_mean, q_elec_mean, irr_mean)

4.490321126760563 3.9253615973268277 164.19859154929577


In [718]:
max_iter=100
tol=1e-6

x0=10
y0=1

irr_mean = 205.10526315789474
q_elec_mean = 43298.14128362253/A_th
#def newton_raphson_2d(x0, y0, tol=1e-6, max_iter=100):
x = x0
y = y0
iteration = 0

building_id = 'BS'
Q_th = buildings.Heat.loc[buildings.Name==building_id]
while iteration < max_iter:
    # Calculate the function values
    h = A_th* ((x*(T_int-T_ext)- y*irr)-(q_elec*f_el)) # W, hourly heat gains from elec and heat transfer through the envelope
    g = A_th* (x*(T_int-T_th)- y*irr_mean -q_elec_mean*f_el) # W, hourly heat gains from elec and heat transfer through the envelope

    heating_indicator = ((T_ext <= T_th) & (h>=0)) 
    f = h[heating_indicator].sum() - Q_th # only sum when heating is needed
        # Check for convergence
    if abs(h.all()) < tol and abs(g.all()) < tol:
        #return x, y
        break
    # Calculate the Jacobian matrix
    J =A_th * [ [(T_int-T_ext).sum(axis=1), -irr.sum(axis=1)],
                [T_int-T_th, -irr_mean]]

    # Calculate the update step
    delta = [-f, -g]

    # Solve the linear system using matrix inversion
    delta_x, delta_y = [sum(J[i][j] * delta[j] for j in range(2)) for i in range(2)]

    # Update x and y
    x += delta_x
    y += delta_y

    iteration += 1
# Initial guess
x0, y0 = 10.0, 1.0

# Call the Newton-Raphson method
solution = newton_raphson_2d(x0, y0)
print("Roots (x, y):", solution)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().